### Getting symbols (Level 1)

In [1]:
def get_all_pairs():
    try:
        exchange_info = client.get_exchange_info()["symbols"]
    except:
        return None

    # This'll contain all symbols in Binance that has USDT
    list_symbols_USDT = []
    symbols_USDT = ""

    for item in exchange_info:
        symbol = item["symbol"]

        if "USDT" in symbol:
            symbols_USDT += symbol + ","
            list_symbols_USDT.append(symbol)

    symbols_USDT = symbols_USDT[:-1]

    with open("pairs.txt", "w") as file:
        file.write(symbols_USDT.replace('\'','').replace(' ',''))
    
    return list_symbols_USDT

### Filter - percent change in 24hrs

In [2]:
def filter1(symbols):
    try:
        info = client.get_ticker()
    except:
        return None
    
    all_change_pair = {} # All symbols paired with corresponding 24hrs change percentile
    USDT_change_pair = {} # All symbols with USDT paired with corresponding 24hrs change percentile
    level1 = [] # List of symbols with more than x% change within 24hrs

    for item in info:
        all_change_pair[item["symbol"]] = float(item["priceChangePercent"])

    for item in all_change_pair:
        if item in symbols:
            USDT_change_pair[item] = all_change_pair[item]

    # Sort by 24hrs change percentage
    sorted_USDT_change_pair = {
        pair[0]: pair[1]
        for pair in sorted(USDT_change_pair.items(), key=lambda x: x[1], reverse=True)
    }

    for item in sorted_USDT_change_pair.items():
        
        if item[1] > 10.0:
            level1.append(item[0])
            print(item[0],item[1])

    with open("pairs_level_1.txt", "w") as file:
        file.write(str(level1)[1:-1].replace('\'','').replace(' ',''))

    # level1 contains the list of symbols that pass the first criteria
    return level1

In [3]:
# Function for getting price
def get_price(ls):
    avg = (float(ls[1]) + float(ls[4])) / 2
    price = float("{:.4f}".format(avg))
    return price


# Function for getting percent change in 6 hour
def get_percent_change(ls_1, ls_2):
    price_1 = get_price(ls_1)
    price_2 = get_price(ls_2)

    change = (price_2 - price_1) / price_1

    change_in_percent = float("{:.4f}".format(change * 100))

    return change_in_percent

# Function for flitering symbols using threshold
def filter_symbol(level, list_of_symbols, threshold, window):
    from datetime import datetime, timedelta

    selected_symbols = []

    for symbol in list_of_symbols:
        try:
            klines = client.get_historical_klines(
                symbol,
                Client.KLINE_INTERVAL_1MINUTE,
                str(datetime.now() - timedelta(hours=window)),
                str(datetime.now()),
            )
        except:
            continue

        percent_change = get_percent_change(klines[0], klines[-1])

        if percent_change > threshold:
            selected_symbols.append(symbol)
            print(symbol, percent_change)
            
    with open("pairs_level_"+str(level)+".txt", "w") as file:
        file.write(str(selected_symbols)[1:-1].replace('\'','').replace(' ',''))
        
    return selected_symbols

### Import Libraries & Load Environment Variables

In [4]:
import time
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

# Binance Client
client = Client(API_KEY, API_SECRET)

### Main Code

In [ ]:
def start():
    list_symbols_USDT = get_all_pairs()
    print("Level 1 - 24hrs window")
    level1 = filter1(list_symbols_USDT)
    print("\nLevel 2 - 6hrs window")
    level2 = filter_symbol(2, level1, 6, 12) #filter_symbol (level, list_of_symbols, threshold, window)
    print("\nLevel 3 - 1hrs window")
    level3 = filter_symbol(3, level2, 3, 7)
    print('\n Finish iteration\n')

while(True):
    start()

Level 1 - 24hrs window
ADAUPUSDT 30.0
BELUSDT 28.52
LINKUPUSDT 27.817
PEOPLEUSDT 23.56
BLZUSDT 23.402
MITHUSDT 22.086
ONGUSDT 19.472
XECUSDT 19.05
MCUSDT 18.557
BTCUPUSDT 18.339
BALUSDT 18.03
HNTUSDT 16.774
RAMPUSDT 15.979
ENSUSDT 15.952
THETAUSDT 15.905
CTSIUSDT 15.35
LRCUSDT 14.877
ETHUPUSDT 14.437
ADAUSDT 14.378
POWRUSDT 14.001
LUNAUSDT 13.591
SYSUSDT 13.527
DOTUPUSDT 13.439
BNBUPUSDT 13.101
NEOUSDT 13.1
AXSUSDT 12.859
ONTUSDT 12.85
CRVUSDT 12.77
SOLUSDT 12.72
MFTUSDT 12.698
CELRUSDT 12.57
RUNEUSDT 12.504
XTZUSDT 12.388
TRXUPUSDT 12.296
AUTOUSDT 12.236
ARUSDT 11.926
RNDRUSDT 11.748
ELFUSDT 11.716
IMXUSDT 11.628
RAYUSDT 11.525
FORUSDT 11.357
ZRXUSDT 11.351
C98USDT 11.345
MATICUSDT 11.282
LINKUSDT 11.279
STORJUSDT 11.259
AVAXUSDT 11.138
POLYUSDT 11.129
BATUSDT 10.96
GMTUSDT 10.946
GTOUSDT 10.883
OMGUSDT 10.803
MTLUSDT 10.649
RLCUSDT 10.638
KDAUSDT 10.499
SCUSDT 10.428
CAKEUSDT 10.411
SUSHIUSDT 10.361
RENUSDT 10.35
BAKEUSDT 10.345
SNXUSDT 10.322
RSRUSDT 10.234
TOMOUSDT 10.179
DGBUSDT 1